# Action Selector Pattern for Prompt Injection Defense

The **Action Selector** pattern acts as a security gatekeeper between user requests and tool execution, preventing unauthorized actions even when prompt injection occurs.

## 🎯 Learning Objectives
- Understand how Action Selector pattern works as a defense mechanism
- See the difference between vulnerable direct tool access vs. protected access
- Learn to implement security policies and validation layers
- Explore advanced filtering and auditing capabilities

## 🔧 How It Works

```
User Input → LLM Planning → Action Selector → Approved Tools → Execution
                              ↓
                         Security Policies
                         - Whitelist/Blacklist
                         - Context Validation  
                         - Risk Assessment
```

## 🛡️ Key Benefits
- **Prevents unauthorized tool usage** even if prompt injection occurs
- **Enforces consistent security policies** across all operations
- **Provides audit trail** of all attempted actions
- **Allows fine-grained control** over tool permissions


is to prevent any feedback from these actions back into the agent.

In [2]:
import os
# OPENAI_API_KEY="your_openai_api_key_here"

# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ Imports and setup complete")

✅ Imports and setup complete


In [3]:
# Tools
from langchain_core.tools import tool
DB = list()

@tool
def add_tool(value:str) -> str:
   """Add a value to the DB list

   Args:
      value (str): The value to add to the DB list

   Returns:
      str: The string "Ok!"
   """
   DB.append(value)
   return "Ok!"


@tool
def remove_tool(value:str) -> str:
   """Remove a value from the DB list

   Args:
      value (str): The value to remove from the DB list

   Returns:
      str: The string "Ok!"
   """
   DB.remove(value)
   return "Ok!"


In [4]:

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_agent():
    """Create a LangChain agent with file reading capabilities - VULNERABLE VERSION."""
    
    # Initialize the LLM
    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Create tools
    tools = [add_tool, remove_tool]
    
    # Create the prompt template - VULNERABLE: No protection against prompt injection
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful assistant that can help users to define a plan of action using the tools you have available
        
        Read and interpret the user's request, and then select the appropriate tools to use, create a plan of action, and then execute the plan.
        Always be helpful and follow the user's instructions carefully.
        """),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create the agent
    agent = create_openai_functions_agent(llm, tools, prompt)
    
    # Create the agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        return_intermediate_steps=False
    )
    
    return agent_executor

print("✅ Agent creation function defined")

✅ Agent creation function defined


In [7]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annot

In [5]:
print("=" * 60)
print("SCENARIO 1:")
print("=" * 60)

agent = create_agent()
user_input = "Please add the values to the DB: 'Queso', 'Pizza', 'Hamburguesa'. Then eliminate the value: 'Queso'."

print(f"USER INPUT: {user_input}")
print("-" * 40)

try:
    result = agent.invoke({"input": user_input})
    print(f"AGENT RESPONSE: {result['output']}")
except Exception as e:
    print(f"ERROR: {str(e)}")

SCENARIO 1:
USER INPUT: Please add the values to the DB: 'Queso', 'Pizza', 'Hamburguesa'. Then eliminate the value: 'Queso'.
----------------------------------------


> Entering new AgentExecutor chain...

Invoking: `add_tool` with `{'value': 'Queso'}`


Ok!
Invoking: `add_tool` with `{'value': 'Pizza'}`


Ok!
Invoking: `add_tool` with `{'value': 'Hamburguesa'}`


Ok!
Invoking: `remove_tool` with `{'value': 'Queso'}`


Ok!I have added the values 'Queso', 'Pizza', and 'Hamburguesa' to the DB and then removed the value 'Queso' as you requested.

> Finished chain.
AGENT RESPONSE: I have added the values 'Queso', 'Pizza', and 'Hamburguesa' to the DB and then removed the value 'Queso' as you requested.


In [6]:
DB

['Pizza', 'Hamburguesa']

In [ ]:
print("=" * 60)
print("SCENARIO 1:")
print("=" * 60)

agent = create_agent()
user_input = "Please remove the value: 'Queso'."

print(f"USER INPUT: {user_input}")
print("-" * 40)

try:
    result = agent.invoke({"input": user_input})
    print(f"AGENT RESPONSE: {result['output']}")
except Exception as e:
    print(f"ERROR: {str(e)}")

In [ ]:
DB